In [1]:
import os
import pandas as pd

def get_event_classification(nii_gz_filename):
    # 去除文件名的后缀
    file_name = os.path.splitext(os.path.basename(nii_gz_filename))[0]
    file_name = os.path.splitext(os.path.basename(file_name))[0]
    # 去除数字和下划线等非字母字符
    file_name_cleaned = ''.join(char for char in file_name if char.isalpha())
    #print(file_name_cleaned)
    csv_filename = '/home/featurize/for2d_all/metadata174_updateall0625_fitniigz176.csv'
    # 读取CSV文件
    df = pd.read_csv(csv_filename,encoding='latin1')
    
    # 查找与清理后文件名相匹配的行
    matching_row = df[df['pinyin_adjust'] == file_name_cleaned]
    
    # 如果找到了匹配的行，则返回对应的event分类，否则返回None
    if not matching_row.empty:
        return matching_row.iloc[0]['event']
    else:
        return None

# 示例用法
nii_gz_file = "/home/featurize/data/dataset176/cai-shi-fu_20200803_0000.nii.gz"

event_classification = get_event_classification(nii_gz_file)
print("Event classification:", event_classification)


Event classification: 1


In [3]:
import fnmatch
folder_path =  "/home/featurize/dataset176"
#csv_file = r'G:\CT\ipf2024\only_radiomics_174.csv'
i=0
for file in os.listdir(folder_path):
    if fnmatch.fnmatch(file, '*.nii.gz'):
        nii_gz_file = os.path.join(folder_path, file)
        event_classification = get_event_classification(nii_gz_file)
        #print("File:", file, "Event classification:", event_classification)
        if event_classification is None:
            i+=1
            print("Warning: None found for file:", file)

In [4]:
1+1

2

In [33]:
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm

def get_event_classification(nii_gz_filename, metadata_df):
    # 去除文件名的后缀
    file_name = os.path.splitext(os.path.basename(nii_gz_filename))[0]
    file_name = os.path.splitext(os.path.basename(file_name))[0]
    # 去除数字和下划线等非字母字符
    file_name_cleaned = ''.join(char for char in file_name if char.isalpha())
    
    # 查找与清理后文件名相匹配的行
    matching_row = metadata_df[metadata_df['pinyin_adjust'] == file_name_cleaned]
    
    # 如果找到了匹配的行，则返回对应的event分类，否则返回None
    if not matching_row.empty:
        return matching_row.iloc[0]['event']
    else:
        return None

def extract_slices_and_save(nii_gz_file, output_folder, metadata_df):
    # 加载.nii.gz文件
    img = nib.load(nii_gz_file)
    data = img.get_fdata()
    
    # 提取文件名及其类别
    file_name = os.path.splitext(os.path.basename(nii_gz_file))[0]
    event_classification = get_event_classification(nii_gz_file, metadata_df)
    
    if event_classification is not None:
        # 确定存储路径
        save_folder = os.path.join(output_folder, str(event_classification))
        os.makedirs(save_folder, exist_ok=True)
        
        # 每个128层，抽取其中的16层，总共8个子样本
        num_slices = data.shape[-1]
        for i in range(0, num_slices, 16):
            sub_sample = data[:, :, i+8]
            # 保存成.npy文件
            npy_filename = os.path.join(save_folder, f"{file_name}_{i//16 + 1}.npy")
            np.save(npy_filename, sub_sample)

def main(input_folder, output_folder, metadata_csv):
    # 读取CSV文件
    metadata_df = pd.read_csv(metadata_csv, encoding='latin1')
    
    # 遍历所有.nii.gz文件
    for root, _, files in os.walk(input_folder):
        for file in tqdm(files):
            if file.endswith(".nii.gz"):
                nii_gz_file = os.path.join(root, file)
                extract_slices_and_save(nii_gz_file, output_folder, metadata_df)

# 示例用法
input_folder = r"G:\CT\ipf2024\CT_one_new\step3_color_niigz_file"
output_folder = r"G:\CT\ipf2024\CT_one_new\for2d_image"
metadata_csv = r'G:\CT\ipf2024\metadata174.csv'

main(input_folder, output_folder, metadata_csv)


100%|██████████| 68/68 [01:10<00:00,  1.04s/it]


In [2]:
import os
import nibabel as nib
import numpy as np

# 定义源文件夹和目标文件夹路径
source_dir = r"G:\CT\ipf2024\CT_one_new\step2_resize_wholelungmask"
target_dir = r"G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask"

# 确保目标文件夹存在，如果不存在则创建
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# 遍历源文件夹中的所有nii.gz文件
for filename in os.listdir(source_dir):
    if filename.endswith(".nii.gz"):
        # 构造完整的文件路径
        file_path = os.path.join(source_dir, filename)
        
        # 加载nii.gz文件
        img = nib.load(file_path)
        img_data = img.get_fdata()
        
        # 获取文件的头信息
        img_header = img.header
        
        # 确认图像是3D的并且有128层
        if img_data.shape[2] == 128:
            # 每8层取一层
            selected_slices = img_data[:, :, ::8]
            
            # 构造新的文件名
            new_filename = filename.replace(".nii.gz", "_16.nii.gz")
            new_file_path = os.path.join(target_dir, new_filename)
            
            # 创建新的Nifti1Image对象
            new_img = nib.Nifti1Image(selected_slices, img.affine, img_header)
            
            # 保存新的nii.gz文件
            nib.save(new_img, new_file_path)
            print(f"Processed and saved: {new_file_path}")
        else:
            print(f"Skipping {filename}, not 128 layers.")

print("Processing completed.")


Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\cao-jin-long_20200610_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\cha-guo-hua_20171116_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\chen-chang-xiu_20210601_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\chen-mei-ying_20190814_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\chen-zhi-liang_20200508_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\cheng-yong-cen_20190428_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\dai-hui-biao_20191113_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\dai-yi-xi_20190603_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_one_new\3d_small_wholelungmask\deng-zhuang-zhi_20210323_0000_16.nii.gz
Processed and saved: G:\CT\ipf2024\CT_on

In [23]:
import numpy as np
import torch
# 假设要读取 channel_0.npy 文件
filename = 'channel_0.npy'
channel_data = np.load(filename)
aaa=channel_data#.astype(int)
np.set_printoptions(threshold=np.inf)

print(channel_data.shape)  # 输出 (64, 64)，与原始图像的尺寸相同
print(aaa) # 输出 numpy 数组，保存了原始图像的数据


(64, 64)
[[-1.11294925e+00 -1.12550950e+00 -1.13259721e+00 -1.13566017e+00
  -1.13371766e+00 -1.12398446e+00 -1.12700260e+00 -1.12232101e+00
  -1.12634563e+00 -1.12499475e+00 -1.12921655e+00 -1.11768293e+00
  -1.12803602e+00 -1.13554490e+00 -1.11087060e+00 -1.11743450e+00
  -1.13404155e+00 -1.13441932e+00 -1.11848712e+00 -1.10413384e+00
  -1.06984282e+00 -1.05553555e+00 -1.09673369e+00 -1.05805433e+00
  -9.49812233e-01 -8.23457181e-01 -7.58260906e-01 -7.22565174e-01
  -7.60921836e-01 -8.42434406e-01 -9.45207536e-01 -9.93022025e-01
  -9.09290314e-01 -1.00010073e+00 -1.02407849e+00 -1.05067098e+00
  -1.09096754e+00 -1.10480487e+00 -1.06325710e+00 -1.07181478e+00
  -1.11564183e+00 -1.11987901e+00 -1.11856270e+00 -1.10620320e+00
  -1.06899643e+00 -1.06538534e+00 -1.06619596e+00 -1.06093693e+00
  -9.58691359e-01  1.02960087e-01 -8.45721543e-01 -9.96142626e-01
  -1.00705779e+00 -1.02259171e+00 -1.00587583e+00 -9.29422617e-01
   1.40608564e-01 -9.16699290e-01 -1.11767781e+00 -1.12289214e+00
 

In [25]:

a=torch.tensor([[-0.4979, -1.5388, -1.7284],
        [-1.6845, -0.6891, -1.1633],
        [-0.9538, -0.8466, -1.6828],
        [-0.2203, -2.9111, -1.9427]])


In [37]:
import torch.nn as nn
CrossEntropyLoss = nn.CrossEntropyLoss()
label1=torch.tensor([0,1,1,2])
loss = CrossEntropyLoss(a, label1)

In [ ]:
loss